In [ ]:
import WesBot as wb
import HandOddsCalcWes as hoc

import texasholdem as th
import texasholdem.evaluator as eval
import numpy as np
import pandas as pd
import random
from tqdm import trange

In [ ]:
class Round:
  def __init__(self):
    self.decisions = []
    self.outcome = int(0)
  
  def add_decision(self, decision:list):
    self.decisions.append(decision)
  
  def close_round(self, outcome:int):
    self.outcome = outcome
  
  def get_decisions(self):
    return self.decisions
  
  def get_one_decision(self, index:int):
    return self.decisions[index]
  
  def get_outcome(self):
    return self.outcome
  
  def copy(self):
    output = Round()
    for decision in self.decisions:
      output.add_decision(decision)
    output.close_round(self.outcome)
    return output

In [3]:
# smort_decision(our_bot:PokerBot, game:th.TexasHoldEm)
# actually make a goddamn decision using my scuffed bot thingy, and record
# all the necessary information for Shro' to use in his thing
# INPUTS:
#   our_bot: a PokerBot object, supposed to be intelligent or something
#   game:    a TexasHoldEm game object that we're playing
# OUTPUT:
#   a list containing:
#     [hand_phase:str,
#      my_hand:list of th.Card,
#      board:list of th.Card,
#      EV:float,
#      my_chips_betting:int,
#      their_chips_betting:int,
#      my_decision:Decision]
# SIDE EFFECT:
#   makes the decision in the TexasHoldEm game object passed in
def smort_decision(our_bot:wb.PokerBot, game:th.TexasHoldEm):
  # get information regarding our hand, the game board, our win prob,
  # their chips bet, and our EV
  win_prob, my_hand_odds = \
    hoc.estimate_win_and_hand_probs(game, game.current_player, 2, 1000)
  loss_prob = 1 - win_prob
  my_chips_betting = game.player_bet_amount(0) + game.chips_to_call(0)
  their_chips_betting = game.player_bet_amount(1)
  EV = win_prob*their_chips_betting - loss_prob*my_chips_betting
  # get features for Shro'
  my_decision = our_bot.make_decision(EV, my_hand_odds, game)
  my_hand = game.get_hand(game.current_player)
  board = []
  for card in game.board:
    board.append(card)
  # record hand phase
  hand_phase = game.hand_phase.name
  # actually make the decision
  if (my_decision.type == "RAISE"):
    game.take_action(th.ActionType.RAISE, my_decision.size)
  elif (my_decision.type == "CALL/CHECK"):
    if (game.validate_move(action = th.ActionType.CALL)):
      game.take_action(th.ActionType.CALL)
    else:
      game.take_action(th.ActionType.CHECK)
  elif (my_decision.type == "FOLD"):
    game.take_action(th.ActionType.FOLD)
  else:
    game.take_action(th.ActionType.ALL_IN)
  output = [hand_phase, my_hand, board, EV, my_chips_betting, their_chips_betting, my_decision]
  return output

# baby_decision(game:th.TexasHoldEm)
# make a random decision for the opponent, just to get someone to play against
# INPUTS:
#   game: a th.TexasHoldEm object that we're playing
# SIDE EFFECT:
#   makes the random decision in the game object passed in
def baby_decision(game:th.TexasHoldEm):
  # opponent makes random decision (reused baby code)
  babys_decision = np.random.choice(3)
  if (babys_decision == 0):
    # baby will call/check if possible
    if (game.validate_move(action = th.ActionType.CALL) or
        game.validate_move(action = th.ActionType.CHECK)):
      decision = wb.Decision("CALL/CHECK")
    elif (game.validate_move(action = th.ActionType.ALL_IN)):
      decision = wb.Decision("ALLIN")
    else:
      decision = wb.Decision("FOLD")
  elif (babys_decision == 1):
    # baby will fold
    decision = wb.Decision("FOLD")
  else:
    # baby will raise if possible
    min_raise = game.get_available_moves().raise_range.start
    max_raise = int(np.min([game.players[game.current_player].chips,
                            game.get_available_moves().raise_range.stop]))
    if (min_raise <= max_raise and
        game.validate_move(action = th.ActionType.RAISE, value = min_raise) and
        game.validate_move(action = th.ActionType.RAISE, value = max_raise)):
      decision = \
        wb.Decision("RAISE", int(np.random.uniform(min_raise, max_raise)))
    elif (game.validate_move(action = th.ActionType.CALL) or
          game.validate_move(action = th.ActionType.CHECK)):
      decision = wb.Decision("CALL/CHECK")
    elif (game.validate_move(action = th.ActionType.ALL_IN)):
      decision = wb.Decision("ALLIN")
    else:
      decision = wb.Decision("FOLD")
  # actually make the decision
  if (decision.type == "RAISE"):
    game.take_action(th.ActionType.RAISE, decision.size)
  elif (decision.type == "CALL/CHECK"):
    if (game.validate_move(action = th.ActionType.CALL)):
      game.take_action(th.ActionType.CALL)
    else:
      game.take_action(th.ActionType.CHECK)
  elif (decision.type == "FOLD"):
    game.take_action(th.ActionType.FOLD)
    return 0
  else:
    game.take_action(th.ActionType.ALL_IN)
  return 1

In [4]:
game = th.TexasHoldEm(buyin=500, big_blind=5, small_blind=2, max_players=2)
game.start_hand()
current_round = Round()
our_bot = wb.PokerBot(k=10, EV_weight=10.0, maturity=50)

epochs = 1000

rounds_list = [] # a list of Round objects

for i in trange(epochs):

  if (game.is_hand_running()):
    # control flow time! we are player 0.
    if (game.current_player == 0):
      # make decision and record decision
      our_decision = smort_decision(our_bot, game)
      current_round.add_decision(our_decision)
      if (our_decision[6].type == "FOLD"):
        who_won = 1
    else:
      # the opponent (who is baby) makes a decision
      who_won = baby_decision(game)
    
    # get the hypothetical winner (in case this is the showdown)

  # ensure a game and hand is running
  if (not game.is_game_running()):
    game = th.TexasHoldEm(buyin=500, big_blind=5, small_blind=2, max_players=2)
    game.start_hand()
    current_round = Round()
  # start a new hand if needed
  if (not game.is_hand_running()):
    current_round.close_round(game._get_last_pot().amount*((-1)**who_won))
    rounds_list.append(current_round.copy())
    game.start_hand()
    current_round = Round()

100%|██████████| 1000/1000 [00:44<00:00, 22.23it/s]


# AFTER THIS POINT, NOTHING IS IN USE AND ONLY REMAINS FOR REFERENCE.

In [ ]:
print(game.hand_phase.name)

In [ ]:
print(game.current_player)

In [ ]:
# get information regarding our hand, the game board, our win prob, their chips bet, and our EV
win_prob, my_hand_odds = hoc.estimate_win_and_hand_probs(game, 0, 2, 1000)
loss_prob = 1 - win_prob
my_chips_betting = game.player_bet_amount(0) + game.chips_to_call(0)
their_chips_betting = game.player_bet_amount(1)
EV = win_prob*their_chips_betting - loss_prob*my_chips_betting
print(game.get_hand(0), game.board, win_prob, their_chips_betting, EV)

In [ ]:
# get decision
my_decision = our_bot.make_decision(EV, my_hand_odds, game)
print(my_decision.type, my_decision.size)

In [ ]:
# actually make decision
if (my_decision.type == "RAISE"):
  game.take_action(th.ActionType.RAISE, my_decision.size)
elif (my_decision.type == "CALL/CHECK"):
  if (game.validate_move(action = th.ActionType.CALL)):
    game.take_action(th.ActionType.CALL)
  else:
    game.take_action(th.ActionType.CHECK)
elif (my_decision.type == "FOLD"):
  game.take_action(th.ActionType.FOLD)
else:
  game.take_action(th.ActionType.ALL_IN)

In [ ]:
# opponent makes random decision (reused baby code)
babys_decision = np.random.choice(3)
if (babys_decision == 0):
  # baby will call/check if possible
  if (game.validate_move(action = th.ActionType.CALL) or
      game.validate_move(action = th.ActionType.CHECK)):
    decision = wb.Decision("CALL/CHECK")
  elif (game.validate_move(game.current_player, th.ActionType.ALL_IN)):
    decision = wb.Decision("ALLIN")
  else:
    decision = wb.Decision("FOLD")
elif (babys_decision == 1):
  # baby will fold
  decision = wb.Decision("FOLD")
else:
  # baby will raise if possible
  min_raise = game.get_available_moves().raise_range[0]
  max_raise = int(np.min([game.players[game.current_player].chips,
                          game.get_available_moves().raise_range[-1]]))
  if (min_raise <= max_raise and
      game.validate_move(action = th.ActionType.RAISE, value = min_raise) and
      game.validate_move(action = th.ActionType.RAISE, value = max_raise)):
    decision = \
      wb.Decision("RAISE", int(np.random.uniform(min_raise, max_raise)))
  elif (game.validate_move(action = th.ActionType.CALL) or
        game.validate_move(action = th.ActionType.CHECK)):
    decision = wb.Decision("CALL/CHECK")
  elif (game.validate_move(action = th.ActionType.ALL_IN)):
    decision = wb.Decision("ALLIN")
  else:
    decision = wb.Decision("FOLD")
print(decision.type, decision.size)

In [ ]:
# actually make decision
if (decision.type == "RAISE"):
  game.take_action(th.ActionType.RAISE, decision.size)
elif (decision.type == "CALL/CHECK"):
  if (game.validate_move(action = th.ActionType.CALL)):
    game.take_action(th.ActionType.CALL)
  else:
    game.take_action(th.ActionType.CHECK)
elif (decision.type == "FOLD"):
  game.take_action(th.ActionType.FOLD)
else:
  game.take_action(th.ActionType.ALL_IN)